In [ ]:
# only when open in colab
!git init .
!git remote add origin https://github.com/coyote-bergstein/compression-using-neural-networks.git
!git pull origin speech-commands

In [101]:
!pip install pydub

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from source.utils import *
from source.constants import *

In [102]:
# only when open in colab
import importlib
importlib.reload(source.constants)
importlib.reload(source.utils)

<module 'source.utils' from '/content/source/utils.py'>

## Dataset

In [103]:
ds = tfds.load('speech_commands', split='train')


## Audio processing testing

In [104]:
import IPython.display as ipd
from source.utils import *
from source.constants import *


signals = []

for example in ds.take(5):
  audio_np = example["audio"].numpy()
  audio_np = audio_np/audio_np.max()
  signals.append(audio_np)


for signal in signals:
  S = generate_spectrogram(signal, HOP_LENGTH, N_FFT)
  reconstructed_signal = reconstruct_audio(S, HOP_LENGTH, N_FFT)

  print("Original Audio:")
  ipd.display(ipd.Audio(signal, rate=AUDIO_SAMPLE_RATE))

  print("Reconstructed Audio:")
  ipd.display(ipd.Audio(reconstructed_signal, rate=AUDIO_SAMPLE_RATE))

print(N_FFT, HOP_LENGTH)

Original Audio:


Reconstructed Audio:


Original Audio:


Reconstructed Audio:


Original Audio:


...

Original Audio:


Reconstructed Audio:


Original Audio:


Reconstructed Audio:


512 128
